In [1]:
#!pip install efficientnet_pytorch

In [1]:
import torch, time, copy
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, models, transforms 
import os 
from PIL import Image
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
import torch.nn.functional as F

In [2]:
image_size = (256,256)
data_transforms={
                        "train":    transforms.Compose
                                            (
                                                [
                                                    transforms.Resize(image_size),
                                                    transforms.ColorJitter(hue=.05, saturation=.05),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.RandomRotation(20, resample=Image.BILINEAR),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                                ]
                                            ),

                        "val":      transforms.Compose
                                            (
                                                [
                                                    transforms.Resize(image_size),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                                ]
                                            )
                }

 
data_dir=r"Dataset/"
 
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ["train", "val"]}

dataloaders            =   {}
dataloaders["train"]   =   torch.utils.data.DataLoader(image_datasets["train"], batch_size=32, shuffle=True, num_workers=2) 
dataloaders["val"]     =   torch.utils.data.DataLoader(image_datasets["val"], batch_size=32, shuffle=False, num_workers=2) 

dataset_sizes={x: len(image_datasets[x]) for x in ["train","val"]}
 
class_names=image_datasets["train"].classes
print(class_names)

num_classes=len(class_names)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
val = {"loss":[],"acc":[]}
train = {"loss":[],"acc":[]}
 


['Capped', 'NoCap']


C:\Users\dobri\Anaconda3\envs\HuggingFace Torch\lib\site-packages\torchvision\transforms\transforms.py:1200: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


In [3]:
def train_model(model, criterion, optimizer , num_epochs=10):
    start_time=time.time()
 
    best_acc= 0.0
 
    for epoch in range(num_epochs):
        print("epoch{}/{}".format(epoch,num_epochs-1))
        print("-"*10)
 
        for phase in ["train", "val"]:
            if phase =="train":
                model.train()
            else:
                model.eval()
 
            running_loss=0.0
            running_corrects=0.0
 
            for inputs,labels in tqdm(dataloaders[phase]):
                inputs=inputs.to(device)
                labels=labels.to(device)
 
                optimizer.zero_grad()
 
                with torch.set_grad_enabled(phase=="train"):
                    outputs=model(inputs)
                    _,preds=torch.max(outputs,1)
                    loss=criterion(outputs,labels)
 
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
 
                running_loss+=loss.item() * inputs.size(0)
                running_corrects+=  torch.sum(preds==labels.data)
 
            epoch_loss=running_loss/dataset_sizes[phase]
            epoch_acc=running_corrects.double()/dataset_sizes[phase]
 
            if phase == "train":
              train["loss"].append(epoch_loss)
              train["acc"].append(epoch_acc.item())
            else:
              val["loss"].append(epoch_loss)
              val["acc"].append(epoch_acc.item())
 
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase,epoch_loss,epoch_acc))
 
            if phase == "val" and epoch_acc>best_acc:
                torch.save(model,"./models/mobilenet-v2-best.pth")
                best_acc=epoch_acc
 
    time_elapsed=time.time()- start_time
    print("training completed in {:.0f}m {:.0f}s".format(time_elapsed//60,time_elapsed%60))
    print("best val accuracy: {:4f}".format(best_acc))
 
    return model
 


In [8]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'mobilenet_v2', pretrained=True)
model.to(device)

Using cache found in C:\Users\dobri/.cache\torch\hub\pytorch_vision_v0.9.0


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)

model_ft = train_model(model, criterion, optimizer, num_epochs=15)
 
torch.save(model_ft,"./models/mobilenet-v2-last.pth")

  0%|          | 0/30 [00:00<?, ?it/s]

epoch0/14
----------


  0%|          | 0/8 [00:00<?, ?it/s]

train Loss: 0.8900 Acc: 0.8977


  0%|          | 0/30 [00:00<?, ?it/s]

...

val Loss: 0.0016 Acc: 1.0000
epoch14/14
----------


  0%|          | 0/8 [00:00<?, ?it/s]

train Loss: 0.0097 Acc: 0.9957


100%|██████████| 8/8 [00:02<00:00,  3.32it/s]

val Loss: 0.0005 Acc: 1.0000
training completed in 4m 58s
best val accuracy: 1.000000


In [4]:
model = torch.load("models/mobilenet-v2-best.pth", map_location = device)

In [5]:
print(model)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen